In [17]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats


In [18]:
energy = pd.read_csv("energy_dataset.csv")
energy

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


# Converting Date time columns

In [19]:
energy = energy.dropna(subset=['time'])

# Convert to datetime with explicit format
try:
    energy['Datetime'] = pd.to_datetime(energy['time'], format='%Y-%m-%d %H:%M:%S%z', utc=True)
except Exception as e:
    print(f"An error occurred: {e}")

energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 30 columns):
 #   Column                                       Non-Null Count  Dtype              
---  ------                                       --------------  -----              
 0   time                                         35064 non-null  object             
 1   generation biomass                           35045 non-null  float64            
 2   generation fossil brown coal/lignite         35046 non-null  float64            
 3   generation fossil coal-derived gas           35046 non-null  float64            
 4   generation fossil gas                        35046 non-null  float64            
 5   generation fossil hard coal                  35046 non-null  float64            
 6   generation fossil oil                        35045 non-null  float64            
 7   generation fossil oil shale                  35046 non-null  float64            
 8   generation fossil peat    

In [20]:
energy['Datetime'] = pd.to_datetime(energy['time'], errors='coerce', utc=True)

# Check conversion
print(energy[['time', 'Datetime']].head())

# Extract various components from 'Datetime'
energy['Year'] = energy['Datetime'].dt.year
energy['Month'] = energy['Datetime'].dt.month
energy['Day'] = energy['Datetime'].dt.day
energy['Hour'] = energy['Datetime'].dt.hour
energy['Minute'] = energy['Datetime'].dt.minute
energy['Second'] = energy['Datetime'].dt.second

energy

                        time                  Datetime
0  2015-01-01 00:00:00+01:00 2014-12-31 23:00:00+00:00
1  2015-01-01 01:00:00+01:00 2015-01-01 00:00:00+00:00
2  2015-01-01 02:00:00+01:00 2015-01-01 01:00:00+00:00
3  2015-01-01 03:00:00+01:00 2015-01-01 02:00:00+00:00
4  2015-01-01 04:00:00+01:00 2015-01-01 03:00:00+00:00


,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,total load actual,price day ahead,price actual,Datetime,Year,Month,Day,Hour,Minute,Second
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,25385.0,50.10,65.41,2014-12-31 23:00:00+00:00,2014,12,31,23,0,0
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,24382.0,48.10,64.92,2015-01-01 00:00:00+00:00,2015,1,1,0,0,0
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,22734.0,47.33,64.48,2015-01-01 01:00:00+00:00,2015,1,1,1,0,0
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,21286.0,42.27,59.32,2015-01-01 02:00:00+00:00,2015,1,1,2,0,0
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,20264.0,38.41,56.04,2015-01-01 03:00:00+00:00,2015,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31 19:00:00+01:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,0.0,...,30653.0,68.85,77.02,2018-12-31 18:00:00+00:00,2018,12,31,18,0,0
35060,2018-12-31 20:00:00+01:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,0.0,...,29735.0,68.40,76.16,2018-12-31 19:00:00+00:00,2018,12,31,19,0,0
35061,2018-12-31 21:00:00+01:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,0.0,...,28071.0,66.88,74.30,2018-12-31 20:00:00+00:00,2018,12,31,20,0,0
35062,2018-12-31 22:00:00+01:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,0.0,...,25801.0,63.93,69.89,2018-12-31 21:00:00+00:00,2018,12,31,21,0,0


In [21]:
energy.drop(columns=['Minute' , 'Second'] , inplace=True)

In [22]:
cols = ['Year', 'Month', 'Day', 'Hour'] + [col for col in energy.columns if col not in ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']]
energy = energy[cols]
energy.drop(columns=['time' , 'Datetime'] , inplace=True)
energy

/var/folders/cf/47mgd2gx2r17z1vchly_bhd00000gn/T/ipykernel_64269/323445904.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy.drop(columns=['time' , 'Datetime'] , inplace=True)


,Year,Month,Day,Hour,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014,12,31,23,447.0,329.0,0.0,4844.0,4821.0,162.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015,1,1,0,449.0,328.0,0.0,5196.0,4755.0,158.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015,1,1,1,448.0,323.0,0.0,4857.0,4581.0,157.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015,1,1,2,438.0,254.0,0.0,4314.0,4131.0,160.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015,1,1,3,428.0,187.0,0.0,4130.0,3840.0,156.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018,12,31,18,297.0,0.0,0.0,7634.0,2628.0,178.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018,12,31,19,296.0,0.0,0.0,7241.0,2566.0,174.0,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018,12,31,20,292.0,0.0,0.0,7025.0,2422.0,168.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018,12,31,21,293.0,0.0,0.0,6562.0,2293.0,163.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


In [23]:
energy.to_excel("test.xlsx")

# Getting report of the dataset

In [ ]:
prof = ProfileReport(energy)
prof.to_file(output_file='energy.html')

In [25]:
energy.isnull().sum()

Year                                               0
Month                                              0
Day                                                0
Hour                                               0
generation biomass                                19
generation fossil brown coal/lignite              18
generation fossil coal-derived gas                18
generation fossil gas                             18
generation fossil hard coal                       18
generation fossil oil                             19
generation fossil oil shale                       18
generation fossil peat                            18
generation geothermal                             18
generation hydro pumped storage aggregated     35064
generation hydro pumped storage consumption       19
generation hydro run-of-river and poundage        19
generation hydro water reservoir                  18
generation marine                                 19
generation nuclear                            

# Cleaning up the data

## Dropping columns
Too many missing values<br>
1) generation hydro pumped storage aggregated<br>
2) forecast wind offshore eday ahead<br>

These all have constant values<br>
3) generation fossil oil shale<br>
4) generation fossil peat<br>
5) generation geothermal<br>
6) generation marine<br>
7) generation wind offshore<br>


In [26]:
df = energy.copy()

In [27]:
df.drop(columns=['generation hydro pumped storage aggregated' , 'forecast wind offshore eday ahead' , 'generation fossil oil shale' , 'generation fossil peat' , 'generation geothermal' , 'generation marine' , 'generation wind offshore'] , inplace=True)

In [28]:
df.drop( columns=['generation fossil coal-derived gas'] ,  inplace=True)
df 

,Year,Month,Day,Hour,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014,12,31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,...,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015,1,1,0,449.0,328.0,5196.0,4755.0,158.0,920.0,...,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015,1,1,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,...,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015,1,1,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,...,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015,1,1,3,428.0,187.0,4130.0,3840.0,156.0,1826.0,...,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018,12,31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,...,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
35060,2018,12,31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,...,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
35061,2018,12,31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,...,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
35062,2018,12,31,21,293.0,0.0,6562.0,2293.0,163.0,108.0,...,93.0,31.0,287.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89


In [ ]:
prof = ProfileReport(df)
prof.to_file(output_file='energy2.html')

## Dropping all null columns because they are less than 0.1% of data

In [29]:
df.dropna(inplace=True)

In [30]:
df1 = df
# Create a row with 0-based indexing
index_row = pd.DataFrame([range(len(df.columns))], columns=df1.columns)

# Insert the row at the beginning of the DataFrame
df1 = pd.concat([index_row, df1], ignore_index=True)
df1.to_excel("energy.xlsx")
df1

,Year,Month,Day,Hour,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,0,1,2,3,4.0,5.0,6.0,7.0,8.0,9.0,...,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.00,23.00
1,2014,12,31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,...,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
2,2015,1,1,0,449.0,328.0,5196.0,4755.0,158.0,920.0,...,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
3,2015,1,1,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,...,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
4,2015,1,1,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,...,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35014,2018,12,31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,...,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
35015,2018,12,31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,...,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
35016,2018,12,31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,...,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
35017,2018,12,31,21,293.0,0.0,6562.0,2293.0,163.0,108.0,...,93.0,31.0,287.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89


# Predicting price

### Shifting price actual column by one so we can create a predict column

In [31]:
df['predict'] = df['price actual'].shift(-1)
df

,Year,Month,Day,Hour,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,...,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,predict
0,2014,12,31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,...,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,64.92
1,2015,1,1,0,449.0,328.0,5196.0,4755.0,158.0,920.0,...,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92,64.48
2,2015,1,1,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,...,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48,59.32
3,2015,1,1,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,...,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32,56.04
4,2015,1,1,3,428.0,187.0,4130.0,3840.0,156.0,1826.0,...,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04,53.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018,12,31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,...,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02,76.16
35060,2018,12,31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,...,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16,74.30
35061,2018,12,31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,...,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30,69.89
35062,2018,12,31,21,293.0,0.0,6562.0,2293.0,163.0,108.0,...,31.0,287.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89,69.88


## dropping last column

In [32]:
df = df.iloc[:-1 , :]
df

,Year,Month,Day,Hour,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,...,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,predict
0,2014,12,31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,...,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,64.92
1,2015,1,1,0,449.0,328.0,5196.0,4755.0,158.0,920.0,...,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92,64.48
2,2015,1,1,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,...,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48,59.32
3,2015,1,1,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,...,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32,56.04
4,2015,1,1,3,428.0,187.0,4130.0,3840.0,156.0,1826.0,...,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04,53.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2018,12,31,17,293.0,0.0,7593.0,2604.0,178.0,1.0,...,266.0,289.0,2952.0,300.0,3028.0,29690.0,29592.0,68.58,77.61,77.02
35059,2018,12,31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,...,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02,76.16
35060,2018,12,31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,...,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16,74.30
35061,2018,12,31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,...,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30,69.89


# adding/dropping different columns to experiment

In [90]:
drop_cols = ['total load forecast' , 'total load actual' , 'price day ahead' , 'price actual' , 'Year' , 'Month' , 'Day', 'Hour']
X = df.drop(columns=drop_cols)
y = df['predict']

In [94]:
train_size = 0.7
split_point = int(len(df) * train_size)
x_train, x_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

In [92]:
# x_train,x_test,y_train,y_test = train_test_split(X, df['predict'],
#                                                  test_size=0.25,
#                                                 random_state=42)

In [95]:
x_train

,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,predict
0,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,64.92
1,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,64.48
2,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,59.32
3,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,56.04
4,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,53.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24548,319.0,592.0,10481.0,4955.0,319.0,0.0,437.0,1615.0,5902.0,59.0,108.0,44.0,305.0,7847.0,68.0,7699.0,64.97
24549,318.0,611.0,7836.0,4678.0,318.0,28.0,434.0,743.0,5900.0,55.0,108.0,41.0,307.0,7713.0,9.0,7797.0,59.00
24550,313.0,539.0,6026.0,4156.0,316.0,80.0,423.0,450.0,5904.0,54.0,98.0,41.0,301.0,7622.0,18.0,7598.0,54.96
24551,311.0,553.0,5077.0,3861.0,302.0,529.0,406.0,547.0,5904.0,54.0,95.0,43.0,299.0,7380.0,18.0,7317.0,52.80


## Scaling the data

In [48]:
from sklearn.preprocessing import StandardScaler
scaler_trf = ColumnTransformer([
    ('scale' , StandardScaler() , slice(0,17))
])

## Applying box-cox transform

In [49]:
from sklearn.preprocessing import PowerTransformer
box_cox = PowerTransformer()
box_cox_trf = ColumnTransformer([
    ("box_cox_trf" , box_cox , slice(0,20))
])

## Feature Selection using PCA

In [75]:
from sklearn.decomposition import PCA
pca = PCA(n_components=7)

## Selecting the model

## Pre-Processing Pipleline

<br> This is the pre-processing pipeline which pre-processes the data.
<br> It includes:
<br> * Scaling the data
<br> * Normalizing the data using yeo-johnson transformer
<br> * And feature selection using PCA

In [126]:
preprocessing_pipe = Pipeline(steps=[
    ("scaler_trf" , scaler_trf),
    ("box_cox_trf",box_cox_trf),
    ("pca" , pca), 
])

# Pipelines for Diferrent models

In [133]:
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

## Random forest pipeline

In [134]:
rf_pipe = Pipeline(steps=[
    ('clf', RandomForestRegressor())
])

In [147]:
# rf_final_pipe = make_pipeline(preprocessing_pipe, rf_pipe)

rf_final_pipe = Pipeline(steps=[
    ('preprocessing', preprocessing_pipe),
    ('pipeline-2', rf_pipe)
])

In [136]:
rf_final_pipe.fit(x_train, y_train)

Pipeline(steps=[('pipeline-1',
                 Pipeline(steps=[('scaler_trf',
                                  ColumnTransformer(transformers=[('scale',
                                                                   StandardScaler(),
                                                                   slice(0, 17, None))])),
                                 ('box_cox_trf',
                                  ColumnTransformer(transformers=[('box_cox_trf',
                                                                   PowerTransformer(),
                                                                   slice(0, 20, None))])),
                                 ('pca', PCA(n_components=7))])),
                ('pipeline-2',
                 Pipeline(steps=[('clf', RandomForestRegressor())]))])

In [138]:
y_pred = rf_final_pipe.predict(x_test)

## Metric scores for random forest

In [141]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

In [142]:
# finding r2 score
rf_r2_score = r2_score(y_test, y_pred)
print("r2 score for random forest is: " , rf_r2_score)

# Mean Absolute Error (MAE)
rf_mae = mean_absolute_error(y_test, y_pred)
print("MAE score for random forest is: " , rf_mae)


# Mean Squared Error (MSE)
rf_mse = mean_squared_error(y_test, y_pred)
print("MSE score for random forest is: " , rf_mse)

# Root Mean Squared Error (RMSE)
rf_rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE score for random forest is: " , rf_rmse)


# Explained Variance Score
rf_evs = explained_variance_score(y_test, y_pred)
print("EVS score for random forest is: " , rf_evs)

r2 score for random forest is:  0.791989978576233
MAE score for random forest is:  4.539196944603083
MSE score for random forest is:  31.846927044535494
RMSE score for random forest is:  5.6433081649450525
EVS score for random forest is:  0.8516897073638778


## Finding cross val score for random forest

In [143]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(rf_final_pipe,x_train,y_train,scoring='r2'))

0.6342821154585436

## Using GridSearchCv to find best parameters for Random Forest

In [148]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'pipeline-2__clf__n_estimators': [50, 100],  # Number of trees in the forest
    'pipeline-2__clf__max_depth': [10, 20],      # Maximum depth of the tree
    'pipeline-2__clf__min_samples_split': [2, 5],# Minimum number of samples required to split a node
    'pipeline-2__clf__min_samples_leaf': [2, 4]  # Minimum number of samples required to be at a leaf node
}

# Set up the GridSearchCV
grid_search = GridSearchCV(rf_final_pipe, param_grid, cv=5, scoring='r2')

# Fit the GridSearchCV to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and the best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Parameters: {'pipeline-2__clf__max_depth': 20, 'pipeline-2__clf__min_samples_leaf': 2, 'pipeline-2__clf__min_samples_split': 2, 'pipeline-2__clf__n_estimators': 100}
Best Cross-Validation Score: 0.6382692188284874


# KNN

In [151]:
knn_pipe = Pipeline(steps=[
    ('clf', KNeighborsRegressor())
])

In [152]:
knn_final_pipe = Pipeline(steps=[
    ('preprocessing', preprocessing_pipe),
    ('pipeline-2', knn_pipe)
])

In [153]:
knn_final_pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler_trf',
                                  ColumnTransformer(transformers=[('scale',
                                                                   StandardScaler(),
                                                                   slice(0, 17, None))])),
                                 ('box_cox_trf',
                                  ColumnTransformer(transformers=[('box_cox_trf',
                                                                   PowerTransformer(),
                                                                   slice(0, 20, None))])),
                                 ('pca', PCA(n_components=7))])),
                ('pipeline-2',
                 Pipeline(steps=[('clf', KNeighborsRegressor())]))])

In [154]:
y_pred = knn_final_pipe.predict(x_test)

In [155]:
# finding r2 score
knn_r2_score = r2_score(y_test, y_pred)
print("r2 score for KNN is: " , knn_r2_score)

# Mean Absolute Error (MAE)
knn_mae = mean_absolute_error(y_test, y_pred)
print("MAE score for KNN is: " , knn_mae)


# Mean Squared Error (MSE)
knn_mse = mean_squared_error(y_test, y_pred)
print("MSE score for KNN is: " , knn_mse)

# Root Mean Squared Error (RMSE)
knn_rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE score for KNN is: " , knn_rmse)


# Explained Variance Score
knn_evs = explained_variance_score(y_test, y_pred)
print("EVS score for KNN is: " , knn_evs)

r2 score for KNN is:  0.48322651243106673
MAE score for KNN is:  7.228203312392918
MSE score for KNN is:  79.11949359223301
RMSE score for KNN is:  8.894913917078288
EVS score for KNN is:  0.6945704787904738


## Cross val score for KNN

In [156]:
np.mean(cross_val_score(knn_final_pipe,x_train,y_train,scoring='r2'))

0.40811854486023724

## Using GridSearchCv to find best parameters for knn

In [157]:

param_grid = {
    'pipeline-2__clf__n_neighbors': [3, 5, 7, 10],
    'pipeline-2__clf__weights': ['uniform', 'distance'],
    'pipeline-2__clf__p': [1, 2]  
}

grid_search = GridSearchCV(knn_final_pipe, param_grid, cv=5, scoring='r2')

grid_search.fit(x_train, y_train)

print("Best parameters found:", grid_search.best_params_)
print("Best r2 score:", grid_search.best_score_)


Best parameters found: {'pipeline-2__clf__n_neighbors': 10, 'pipeline-2__clf__p': 2, 'pipeline-2__clf__weights': 'distance'}
Best r2 score: 0.44572028735442554


# Linear Regression

In [164]:
lr_pipe = Pipeline(steps=[
    ('clf', LinearRegression())
])

In [165]:
lr_final_pipe = Pipeline(steps=[
    ('preprocessing', preprocessing_pipe),
    ('pipeline-2', lr_pipe)
])

In [166]:
lr_final_pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler_trf',
                                  ColumnTransformer(transformers=[('scale',
                                                                   StandardScaler(),
                                                                   slice(0, 17, None))])),
                                 ('box_cox_trf',
                                  ColumnTransformer(transformers=[('box_cox_trf',
                                                                   PowerTransformer(),
                                                                   slice(0, 20, None))])),
                                 ('pca', PCA(n_components=7))])),
                ('pipeline-2', Pipeline(steps=[('clf', LinearRegression())]))])

In [167]:
y_pred = knn_final_pipe.predict(x_test)

## Metrics for Linear Regression Classifier

In [169]:
# finding r2 score
lr_r2_score = r2_score(y_test, y_pred)
print("r2 score for lr is: " , lr_r2_score)

# Mean Absolute Error (MAE)
lr_mae = mean_absolute_error(y_test, y_pred)
print("MAE score for lr is: " , lr_mae)


# Mean Squared Error (MSE)
lr_mse = mean_squared_error(y_test, y_pred)
print("MSE score for lr is: " , lr_mse)

# Root Mean Squared Error (RMSE)
lr_rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE score for lr is: " , lr_rmse)


# Explained Variance Score
lr_evs = explained_variance_score(y_test, y_pred)
print("EVS score for lr is: " , lr_evs)

r2 score for lr is:  0.48322651243106673
MAE score for lr is:  7.228203312392918
MSE score for lr is:  79.11949359223301
RMSE score for lr is:  8.894913917078288
EVS score for lr is:  0.6945704787904738


# SVM

In [170]:
from sklearn.svm import SVR

In [172]:
svm_pipe = Pipeline(steps=[
    ('clf', SVR(kernel='rbf'))
])

In [173]:
svm_final_pipe = Pipeline(steps=[
    ('preprocessing', preprocessing_pipe),
    ('pipeline-2', svm_pipe)
])

In [174]:
svm_final_pipe.fit(x_train, y_train)

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('scaler_trf',
                                  ColumnTransformer(transformers=[('scale',
                                                                   StandardScaler(),
                                                                   slice(0, 17, None))])),
                                 ('box_cox_trf',
                                  ColumnTransformer(transformers=[('box_cox_trf',
                                                                   PowerTransformer(),
                                                                   slice(0, 20, None))])),
                                 ('pca', PCA(n_components=7))])),
                ('pipeline-2', Pipeline(steps=[('clf', SVR())]))])

In [175]:
y_pred = svm_final_pipe.predict(x_test)

In [176]:
# finding r2 score
svm_r2_score = r2_score(y_test, y_pred)
print("r2 score for svm is: " , svm_r2_score)

# Mean Absolute Error (MAE)
svm_mae = mean_absolute_error(y_test, y_pred)
print("MAE score for svm is: " , svm_mae)


# Mean Squared Error (MSE)
svm_mse = mean_squared_error(y_test, y_pred)
print("MSE score for svm is: " , svm_mse)

# Root Mean Squared Error (RMSE)
svm_rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE score for svm is: " , svm_rmse)


# Explained Variance Score
svm_evs = explained_variance_score(y_test, y_pred)
print("EVS score for svm is: " , svm_evs)

r2 score for svm is:  0.8355701180758394
MAE score for svm is:  4.025829302595431
MSE score for svm is:  25.17468349715771
RMSE score for svm is:  5.017437941535272
EVS score for svm is:  0.8925625615190113


## cross val score for SVM

In [179]:
np.mean(cross_val_score(svm_final_pipe,x_train,y_train,scoring='r2'))

0.6338653875767063

# GridSearchCV

In [177]:
param_grid = {
    'pipeline-2__clf__C': [0.1, 1, 10],
    'pipeline-2__clf__epsilon': [0.01, 0.1, 1],
    'pipeline-2__clf__kernel': ['linear', 'rbf', 'poly'] 
}

grid_search = GridSearchCV(svm_final_pipe, param_grid, cv=5, scoring='r2')

grid_search.fit(x_train, y_train)

print("Best parameters found:", grid_search.best_params_)
print("Best r2 score:", grid_search.best_score_)


Best parameters found: {'pipeline-2__clf__C': 10, 'pipeline-2__clf__epsilon': 1, 'pipeline-2__clf__kernel': 'linear'}
Best r2 score: 0.7093389969666244
